In [1]:
#套件區
import pymssql
import pandas
import csv
import numpy as np
from datetime import datetime
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
#from bokeh.sampledata.stocks import AAPL, GOOG, IBM, MSFT
from bokeh.models.widgets import Tabs, Panel

In [17]:
#設定區
CONFIG={
    "server":"server",
    "username":"username",
    "password":"password",
    "table":"table",
}

In [18]:
#SQL連線區
conn = pymssql.connect(CONFIG["server"], CONFIG["username"], CONFIG["password"], CONFIG["table"])
cursor = conn.cursor()

In [75]:
#爬台銀牌告匯率區
stdate= datetime(2016, 5, 16, 23, 9, 12, 946118).date()
st=stdate.strftime('%Y-%m-%d')
enddata=(datetime.now().date()+timedelta(days=1)).strftime('%Y-%m-%d')
def getdatas(time):
    import pandas
    dfs=pandas.read_html('http://rate.bot.com.tw/xrt/all/%s' %time )
    currency=dfs[0]
    currency=currency.ix[:,0:5]
    currency.columns=[u'type',u'cash-in',u'cash-out',u'sell-in',u'sell-out']
    currency
    currency[u'type']=currency[u'type'].str.extract('\((\w+)\)')
    return currency
while (True):
    if(st==enddata):
        break;
    dfs=getdatas(st)
    for i in range(len(dfs)):
        cursor.execute("INSERT INTO [BANK] VALUES (N'%s' ,N'%s',N'%s',N'%s',N'%s',N'%s')" %(dfs[u'type'][i],dfs[u'cash-in'][i],dfs[u'cash-out'][i],dfs[u'sell-in'][i],dfs[u'sell-out'][i],st))
        conn.commit()
    print(st)    
    stdate=stdate+timedelta(days=1)
    st=stdate.strftime('%Y-%m-%d')

c:\users\ext_tomt\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


2016-05-16
2016-05-17
2016-05-18
2016-05-19
2016-05-20
2016-05-21
2016-05-22
2016-05-23
2016-05-24
2016-05-25
2016-05-26
2016-05-27
2016-05-28
2016-05-29
2016-05-30
2016-05-31
2016-06-01
2016-06-02
2016-06-03
2016-06-04
2016-06-05
2016-06-06
2016-06-07
2016-06-08
2016-06-09
2016-06-10
2016-06-11
2016-06-12
2016-06-13
2016-06-14
2016-06-15
2016-06-16
2016-06-17
2016-06-18
2016-06-19
2016-06-20
2016-06-21
2016-06-22
2016-06-23
2016-06-24
2016-06-25
2016-06-26
2016-06-27
2016-06-28
2016-06-29
2016-06-30
2016-07-01
2016-07-02
2016-07-03
2016-07-04
2016-07-05
2016-07-06
2016-07-07
2016-07-08
2016-07-09
2016-07-10
2016-07-11
2016-07-12
2016-07-13
2016-07-14
2016-07-15
2016-07-16
2016-07-17
2016-07-18
2016-07-19
2016-07-20
2016-07-21
2016-07-22
2016-07-23
2016-07-24
2016-07-25
2016-07-26
2016-07-27
2016-07-28
2016-07-29
2016-07-30
2016-07-31
2016-08-01
2016-08-02
2016-08-03
2016-08-04
2016-08-05
2016-08-06
2016-08-07
2016-08-08
2016-08-09
2016-08-10
2016-08-11
2016-08-12
2016-08-13
2016-08-14

In [10]:
#抓出來透過BOHEK作圖表區

def datetimes(x):
    return np.array(x, dtype=np.datetime64)
def dopanel(stype):
    cursor.execute("exec [GetBankInfo] '%s'" %stype)
    row=cursor.fetchall()
    cash_in=[]
    cash_out=[]
    sell_in=[]
    sell_out=[]
    time =[]
    for i in range( len(row)):
        try:
            cash_in.append(float(row[i][1]))
        except ValueError:
            break;
        try: 
            cash_out.append(float(row[i][2]))
        except ValueError:
            break;
        try: 
            sell_in.append(float(row[i][3]))
        except ValueError:
            break;
        try: 
            sell_out.append(float(row[i][4]))
        except ValueError:
            break;
        
        
        time.append(row[i][5])
    p1 = figure(x_axis_type="datetime", title=stype)
    p1.grid.grid_line_alpha=0.3
    p1.xaxis.axis_label = 'Date'
    p1.yaxis.axis_label = 'Price'
    p1.line(datetimes(time), cash_in, color='#A6CEE3', legend='cash_in')
    p1.line(datetimes(time), cash_out, color='#B2DF8A', legend='cash_out')
    p1.line(datetimes(time), sell_in, color='#33A02C', legend='sell_in')
    p1.line(datetimes(time), sell_out, color='#FB9A99', legend='sell_out')
    p1.legend.location = "top_left"
    return p1
tabslist=[];
#for i in range(len(typelist)):
#    ws=Panel(child=dopanel(typelist[i]), title=typelist[i])
#    tabslist.append(ws)
"""
IDR
KRW
MYR
PHP
VND
ZAR
"""
tabslist.append(Panel(child=dopanel('USD'), title='USD'))
tabslist.append(Panel(child=dopanel('CAD'), title='CAD'))
tabslist.append(Panel(child=dopanel('CHF'), title='CHF'))
tabslist.append(Panel(child=dopanel('THB'), title='THB'))
tabslist.append(Panel(child=dopanel('CNY'), title='CNY'))
tabslist.append(Panel(child=dopanel('GBP'), title='GBP'))
tabslist.append(Panel(child=dopanel('AUD'), title='AUD'))
tabslist.append(Panel(child=dopanel('EUR'), title='EUR'))
tabslist.append(Panel(child=dopanel('NZD'), title='NZD'))
tabslist.append(Panel(child=dopanel('SGD'), title='SGD'))
tabslist.append(Panel(child=dopanel('SEK'), title='SEK'))
tabslist.append(Panel(child=dopanel('JPY'), title='JPY'))
tabslist.append(Panel(child=dopanel('HKD'), title='HKD'))


#tabslist.append(Panel(child=dopanel('nan'), title='nan'))
#tabslist.append(Panel(child=dopanel('IDR'), title='IDR'))
#tabslist.append(Panel(child=dopanel('ZAR'), title='ZAR'))
#tabslist.append(Panel(child=dopanel('PHP'), title='PHP'))
#tabslist.append(Panel(child=dopanel('VND'), title='VND'))
#tabslist.append(Panel(child=dopanel('MYR'), title='MYR'))
#tabslist.append(Panel(child=dopanel('KRW'), title='KRW'))

tabs = Tabs(tabs=tabslist)
output_file("Bank.html", title="Bank-Crawl.py example")
show(tabs)